In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
data = pd.read_csv('hasil scraping10.csv')

In [3]:
data = data[['Datetime','Text']]

In [4]:
data

,Datetime,Text
0,2022-05-14 23:47:33+00:00,Pertama benarkah harga air bersih lebih mahal ...
1,2022-05-14 23:40:21+00:00,"Minyak goreng, dll kapan ya turun harga :') \n..."
2,2022-05-14 23:20:52+00:00,@detikcom Turun kan harga minyak goreng uae pa
3,2022-05-14 23:14:02+00:00,Berikut Strategi Supaya Harga Minyak Goreng Sa...
4,2022-05-14 22:48:49+00:00,"@eg_nrs Selamat pagi Mih...\n\nMau tanya Mih,m..."
...,...,...
77952,2020-01-03 08:03:37+00:00,"Waw Promo Superindo Periode 3-5 Januari 2020, ..."
77953,2020-01-02 14:40:33+00:00,Abis mandi udah make skincare siap tidur\nBapa...
77954,2020-01-02 14:09:32+00:00,"@bucinajateros @in_makiy @FOODFESS2 Ya, \nharg..."
77955,2020-01-01 06:49:02+00:00,@Savianisaa Mahal bgt harga minyak goreng segi...


In [5]:
!pip install emoji

In [6]:
!pip install transformers

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import defaultdict

# NLP
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import emoji

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mocha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Case Folding

In [7]:
import re

# Buat fungsi untuk langkah case folding
def casefolding(text):
   # ubah text menjadi huruf kecil
  text = text.lower()
  # ubah enter menjadi spasi
  text = re.sub(r'\n', ' ', text)
  # hapus emoji
  text = emoji.demojize(text)
  text = re.sub(':[A-Za-z_-]+:', ' ', text) # delete emoji
  # hapus emoticon
  text = re.sub(r"([xX;:]'?[dDpPvVoO3)(])", ' ', text)
  # hapus link
  text = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})", "", text)
  # hapus usename
  text = re.sub(r"@[^\s]+[\s]?", ' ', text)
  # hapus hashtag
  text = re.sub(r'#(\S+)', r'\1', text)
  # hapus angka dan beberapa simbol
  text = re.sub('[^a-zA-Z,.?!]+',' ',text)
  # clear spasi
  text = re.sub('[ ]+',' ',text)
  return text

In [8]:
raw_sample = data['Text'].iloc[0]
case_folding = casefolding(raw_sample)

print('Raw data\t: ', raw_sample)
print('Case folding\t: ', case_folding)

Raw data	:  Pertama benarkah harga air bersih lebih mahal dati minyak goreng?, migor paling murah 14 ribu??, 
Kedua anies guberbur bukan presiden, 18 tuntutan itu salah alamat kalau ke pemda DKI,!
@detikcom  yg ngaco atau narasumber. https://t.co/m86erprJw7
Case folding	:  pertama benarkah harga air bersih lebih mahal dati minyak goreng?, migor paling murah ribu??, kedua anies guberbur bukan presiden, tuntutan itu salah alamat kalau ke pemda dki,! yg ngaco atau narasumber. 


# Word Normalization

In [48]:
key_norm = pd.read_csv('https://github.com/louisowen6/NLP_bahasa_resources/blob/master/combined_slang_words.txt')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 28, saw 411


In [9]:
key_norm = pd.read_csv('combined_slang_words.txt')
key_norm

,_id,singkat,hasil
0,NaN,@,di
1,NaN,abis,habis
2,NaN,ad,ada
3,NaN,adlh,adalah
4,NaN,afaik,as far as i know
...,...,...,...
1016,NaN,wkwkkw,tertawa
1017,NaN,ahokncc,ahok
1018,NaN,istaa,nista
1019,NaN,benarjujur,jujur


In [10]:
key_norm['_id '] = np.arange(key_norm.shape[0])
key_norm['_id ']= key_norm['_id '].astype("int32")
def add_1(x):
    return x+1
key_norm['_id '] = key_norm['_id '].add(1)

In [11]:
print(key_norm.dtypes)
print(key_norm)

_id         int32
singkat    object
 hasil     object
dtype: object
      _id         singkat                 hasil
0        1             @                   di  
1        2          abis               habis   
2        3            ad                  ada  
3        4          adlh               adalah  
4        5         afaik     as far as i know  
...    ...            ...                   ...
1016  1017        wkwkkw             tertawa   
1017  1018       ahokncc                 ahok  
1018  1019         istaa                nista  
1019  1020    benarjujur                jujur  
1020  1021         mgkin               mungkin 

[1021 rows x 3 columns]


In [12]:
#key_norm = pd.read_csv('https://github.com/louisowen6/NLP_bahasa_resources/blob/master/combined_slang_words.txt')

def text_normalize(text):
  text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
  text = str.lower(text)
  return text

# Filtering stopword removal

In [13]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')

In [14]:
len(stopwords_ind)

758

In [15]:
# Buat fungsi untuk langkah stopword removal

#more_stopword = ['barang','kirim']                    # Tambahkan kata dalam daftar stopword
#stopwords_ind = stopwords_ind + more_stopword

def remove_stop_words(text):
  clean_words = []
  text = text.split()
  for word in text:
      if word not in stopwords_ind:
          clean_words.append(word)
  return " ".join(clean_words)

In [16]:
raw_sample = data['Text'].iloc[0]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)

Raw data		:  Pertama benarkah harga air bersih lebih mahal dati minyak goreng?, migor paling murah 14 ribu??, 
Kedua anies guberbur bukan presiden, 18 tuntutan itu salah alamat kalau ke pemda DKI,!
@detikcom  yg ngaco atau narasumber. https://t.co/m86erprJw7
Case folding		:  pertama benarkah harga air bersih lebih mahal dati minyak goreng?, migor paling murah ribu??, kedua anies guberbur bukan presiden, tuntutan itu salah alamat kalau ke pemda dki,! yg ngaco atau narasumber. 
Stopword removal	:  harga air bersih mahal dati minyak goreng?, migor murah ribu??, anies guberbur presiden, tuntutan salah alamat pemda dki,! yg ngaco narasumber.


# Stemming

In [17]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Buat fungsi untuk langkah stemming bahasa Indonesia
def stemming(text):
  text = stemmer.stem(text)
  return text

In [18]:
raw_sample = data['Text'].iloc[0]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)
text_stemming = stemming(stopword_removal)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)
print('Stemming\t\t: ', text_stemming)

Raw data		:  Pertama benarkah harga air bersih lebih mahal dati minyak goreng?, migor paling murah 14 ribu??, 
Kedua anies guberbur bukan presiden, 18 tuntutan itu salah alamat kalau ke pemda DKI,!
@detikcom  yg ngaco atau narasumber. https://t.co/m86erprJw7
Case folding		:  pertama benarkah harga air bersih lebih mahal dati minyak goreng?, migor paling murah ribu??, kedua anies guberbur bukan presiden, tuntutan itu salah alamat kalau ke pemda dki,! yg ngaco atau narasumber. 
Stopword removal	:  harga air bersih mahal dati minyak goreng?, migor murah ribu??, anies guberbur presiden, tuntutan salah alamat pemda dki,! yg ngaco narasumber.
Stemming		:  harga air bersih mahal dati minyak goreng migor murah ribu anies guberbur presiden tuntut salah alamat pemda dki yg ngaco narasumber


# Text Preprcessing Pipeline

In [19]:
# Buat fungsi untuk menggabungkan seluruh langkah text preprocessing
def text_preprocessing_process(text):
  text = casefolding(text)
  text = text_normalize(text)
  text = remove_stop_words(text)
  text = stemming(text)
  return text

In [20]:
%%time
data['clean_teks'] = data['Text'].apply(text_preprocessing_process)

# Perhatikan waktu komputasi ketika proses text preprocessing

Wall time: 1h 28min 35s


In [21]:
data

,Datetime,Text,clean_teks
0,2022-05-14 23:47:33+00:00,Pertama benarkah harga air bersih lebih mahal ...,harga air bersih mahal dati minyak goreng migo...
1,2022-05-14 23:40:21+00:00,"Minyak goreng, dll kapan ya turun harga :') \n...",minyak goreng dll ya turun harga terkejoet lho...
2,2022-05-14 23:20:52+00:00,@detikcom Turun kan harga minyak goreng uae pa,turun harga minyak goreng uae pa
3,2022-05-14 23:14:02+00:00,Berikut Strategi Supaya Harga Minyak Goreng Sa...,strategi harga minyak goreng sawit rp liter be...
4,2022-05-14 22:48:49+00:00,"@eg_nrs Selamat pagi Mih...\n\nMau tanya Mih,m...",selamat pagi mih mih mengapa harga minyak gore...
...,...,...,...
77952,2020-01-03 08:03:37+00:00,"Waw Promo Superindo Periode 3-5 Januari 2020, ...",waw promo superindo periode januari diskon per...
77953,2020-01-02 14:40:33+00:00,Abis mandi udah make skincare siap tidur\nBapa...,abis mandi udah make skincare tidur pulang baw...
77954,2020-01-02 14:09:32+00:00,"@bucinajateros @in_makiy @FOODFESS2 Ya, \nharg...",ya harga minyak goreng liter gas kg sedap bung...
77955,2020-01-01 06:49:02+00:00,@Savianisaa Mahal bgt harga minyak goreng segi...,mahal bgt harga minyak goreng segitu sav beli ...


In [22]:
# Simpan data yang telah melalui text preprocessing agar kita tidak perlu menjalankan proses tersebut mulai awal (Opsional)
data.to_csv('clean_data_preprocessing_Proyek akhir11.csv')
#data = pd.read_csv('clean_data.csv')